In [ ]:
import json

#recursive fn for iterting through each structre.
def get_val_json(data,keys):
  if len(keys) == 1:
    return data.get(keys[0])
  else:
    new_key = keys[0]
    if new_key in data:
      return get_val_json(data[new_key],keys[1:])
    else:
      return None

# Load JSON data from the file
with open('alogparams_from_ui1.json', 'r') as json_file:
    j_data = json.load(json_file)

#keys to be extracted
t_type_keys = ['design_state_data', 'target', 'type']
t_name_keys = ['design_state_data', 'target', 'target']

#extract values using the generic function above
type_val = get_val_json(data,t_type_keys)
target_val = get_val_json(data,t_name_keys)

print("Type:", type_val)
print("Target:", target_val)

In [ ]:
#2.importing datasets and Reading the features 
#.. what missing imputation needs to be applied

import pandas as pd
iris_df = pd.read_csv('iris.csv')
# print(iris_df.head())

#extracting feature handling metrics from json ->j_data
feat_hdg_info = j_data['design_state_data']['feature_handling']

#loooing through features and applying missing value imputaions
for feature_name, feature_details in feat_hdg_info.items():
  if feature_name == 'species': #as this feature doesn't contain any Impute values
        continue
  if feature_details['feature_details']['missing_values'] == 'Impute':
    impute_method = feature_details['feature_details']['impute_with']
    impute_value = feature_details['feature_details']['impute_value']

    if impute_method =='Average of values':
      iris_df[feature_name].fillna(iris_df[feature_name].mean(), inplace=True)
    elif impute_method == 'custom':
      iris_df[feature_name].fillna(impute_value, inplace=True)

      

In [ ]:
#3.Computing feature reduction based on input 

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.decomposition import PCA

#generic function for different json files.
def apply_featr_reduc(j_df, reduc_method, sel_features, target_feature=None):
  if reduc_method == "No Reduction":
    reduced_df = j_df[sel_features]

  elif reduc_method == "Corr with Target":
    corr_matrix = j_df[sel_features + [target_feature]].corr()
    sorted_corr = corr_matrix[target_feature].abs().sort_values(ascending=False)
    top_features = sorted_corr[1:].index.tollist()
    reduced_df = j_df[top_features]

  elif reduc_method == "Tree-based":
    X = j_df[sel_features]
    y = j_df[target_feature]

    #performing one-hot encoding as this data contain string values in iris-setosa
    X_encoded = pd.get_dummies(X)

    tree_model = ExtraTreesRegressor(n_estimators=100, random_state=0)
    tree_model.fit(X_encoded,y)

    importances = tree_model.feature_importances_
    indices = importances.argsort()[::-1]

    num_top_features = min(10, len(sel_features))
    top_indices = indices[:num_top_features]

    top_features = X_encoded.columns[top_indices]
    reduced_df = X_encoded[top_features]
  
  elif reduc_method == "PCA":
    n_components = len(sel_features)
    pca = PCA(n_components=n_components)
    reduced_df = pd.Dataframe(pca.fit_transform(j_df[sel_features]), columns=[f'PCA_{i+1}' for i in range(n_components)])
  
  return reduced_df

#loading json file if required .. again
with open('alogparams_from_ui1.json', 'r') as json_file:
    j_data = json.load(json_file)

#selected feature extraction
j_data['design_state_data']['session_info']['dataset'] = 'iris.csv' #change to the json file - not using modified iris. same csv is using
j_df = pd.read_csv(j_data['design_state_data']['session_info']['dataset'])
sel_features = [feature_name for feature_name, feature_info in j_data['design_state_data']['feature_handling'].items() if feature_info['is_selected']]

#reduction method
reduc_method = j_data['design_state_data']['feature_reduction']['feature_reduction_method']

#target features extraction
target_feature = j_data['design_state_data']['target']['target'] if reduc_method == "Corr with Target" or reduc_method == "Tree-based" else None

#APPLYING REDUCTION
reduced_df = apply_featr_reduc(j_df, reduc_method, sel_features, target_feature)

print(reduced_df)
